# Jeux Olympique analyses

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

ModuleNotFoundError: No module named 'xgboost'

## Etude des données

Il s'agit d'un ensemble de données historiques sur les Jeux olympiques modernes, comprenant tous les Jeux d'Athènes 1896 à Rio 2016. Ces données ont été éxtraite à partir de www.sports-reference.com en mai 2018.


Notez que les Jeux d'hiver et d'été ont eu lieu la même année jusqu'en 1992. Après cela, ils ont été échelonnés de telle sorte que les Jeux d'hiver ont lieu sur un cycle de quatre ans à partir de 1994, puis les Jeux d'été en 1996, puis les Jeux d'hiver en 1998, et ainsi de suite. 

### Valeurs manquantes

In [ ]:
df_events = pd.read_csv("athlete_events.csv", index_col='ID')
number_index = df_events.index.size
print(f'Il y à {number_index} entrés.')
df_events.head()

Nous regardons si des données sont manquantes. Nous pouvons noter qu'il y a des valeurs manquantes sur ces colonnes.
   - Âge (9474 valeurs manquantes)
   - Height (60171 valeurs manquantes)
   - Weight (62875 valeurs manquantes)
   - Medal (231333 valeurs manquantes !)
   
Vue que nous avons énormement de lignes concernants `Medals` il n'est pas envisagable de les supprimer, cependant, nous pouvons le faire pour les trois autres.

In [ ]:
df_events.isnull().sum()

In [ ]:
df_events.dropna(inplace=True, subset=['Age', 'Height', 'Weight'])
df_events.isnull().sum()

In [ ]:
print(f'Il y a maintenant {df_events.index.size} entrés, et nous avons perdu {100-(df_events.index.size*100)/number_index}% des données. Cela reste respectable pour faire des analyses dessus.')

### Recherchons si nous avons des valeurs abérantes et expliquons les

Il y a 3 principales valeurs à vérifier. L'âge, la taille et le poids.

L'**âge** va de 11 à 71 ans, ce qui semble être cohérant. Un enfant et une personne agée peuvent participer aux JO sans problème.

La **taille** va de 127 à 226 cm représentant des personnes de petite taille et d'autres de grande. Les grandes tailles peuvent s'expliquer aux prédispositions de chaque sport.

De même pour le **poids** allant de 70 à 214 kg.

In [ ]:
df_events[['Age', 'Height', 'Weight']].describe()

### Fouiller les données pour voir ce qu'on peut en faire

<ins>Nombre d'athlètes par sexe</ins>

En regardant le pourcentage d'hommes et de femmes qui ont participé aux jeux olympiques, on peut voir qu'il y a plus d'hommes que de femmes dans la compétition, tous sports confondus.

In [ ]:
df_events['Sex'].value_counts(normalize=True) * 100

<ins>Caractéristiques physiques</ins>

En faisant une moyenne sur les caracteristiques physiques des sportives des hommes et des femmes qui ont participé aux jeux olympiques, on peut noter que les femmes sont globalement plus jeunes, plus petites et moins lourdes que les hommes.

In [ ]:
df_events[['Sex', 'Age', 'Height', 'Weight']].groupby(['Sex']).mean()

<ins>Nombre de médailles par sexe</ins>

Les femmes ont deux fois moins de médailles que les hommes. Cela peut s'expliquer par le nombre d'athlètes féminines inférieur à celui des hommes et également par le nombre d'épreuves différentes selon le sexe.

In [ ]:
df_events[['Sex', 'Medal']].groupby('Sex').count()

<ins>Nombre de médailles gagnées en fonction du sexe</ins>

Lors des débuts des Jeux Olympiques, les femmes ne gagnaient pas de médailles et il n'y avait pas beaucoup de sports aux Jeux Olympiques. En 2016, il y plus de sports et les femmes ont autant de médailles que les hommes.

In [ ]:
df_events[['Year', 'Sex', 'Medal']].groupby(['Year', 'Sex']).count()

<ins>Réprésentation de l'âge des sportifs</ins>

Les femmes sont un peu plus jeunes que les hommes et deux fois moins nombreuses.

In [ ]:
grouped = df_events.groupby('Sex')
for group in grouped:
  plt.hist(group[1].Age, bins=60,  alpha=.7)

<ins>Nombre total de médailles par pays</ins>

In [ ]:
df_events[['NOC', 'Medal']].groupby('NOC').count()

In [ ]:
df_events[['NOC', 'Medal']].groupby('NOC').count().describe()

<ins>Le sport avec le plus de médailles</ins>

In [ ]:
df2 = df_events.groupby('Sport').count()
df2['Medal'].idxmax()

<ins>Les athlètes ayant gagnés une médaille d'or</ins>

In [ ]:
df_events[df_events['Medal'] == "Gold"]

<ins>Les médianes de la taille et du poids</ins>

In [ ]:
df_events[['Height', 'Weight']].median()

<ins>Nombre de médailles par sexe depuis 1900</ins>

On peut observer plusieurs choses :
 - de 1900 à 1950, il n'y a pas beaucoup de sports
 - les femmes ont moins de médailles
 - En 1990, les jeux olympique d'hiver apparaissent (À prendre en compte lors de l'apprentisage)

In [ ]:
dfH = df_events[(df_events['Medal'] != "No Medal") & (df_events['Sex'] == "M")][['Year', 'Medal']]
dfH = dfH.groupby(['Year']).count()

dfF = df_events[(df_events['Medal'] != "No Medal") & (df_events['Sex'] == "F")][['Year', 'Medal']]
dfF = dfF.groupby(['Year']).count()

plt.scatter(dfH.index, dfH['Medal'])
plt.scatter(dfF.index, dfF['Medal'])
plt.show()

### Chercher à tâtons n'est pas très productif, voyons du coté des outils généraux

Sur cette vue on peut remarquer qu'il y a de plus en plus de femmes qui participent aux jeux olympiques au fur et à mesure des années.

In [ ]:
sns.pairplot(df_events, hue = 'Sex')
plt.show()

De plus, avec la matrice de correlation, on remarque que la taille a une bonne relation avec le poids. Ce qui peut s'expliquer par une bonne condition physique.

In [ ]:
sns.heatmap(df_events.corr())
plt.show()

On remarque même une tendance linéaire entre la taille et le poids avec quelques valeurs abérentes aux extrémités, ce qui pourrait perturber l'apprentissage du futur modèle. On remarque même quelques valeurs abérentes sur l'âge.

In [ ]:
sns.jointplot(data=df_events, x="Height", y="Weight")
plt.show()
sns.boxplot(data=df_events, x="Height")
plt.show()
sns.boxplot(data=df_events, x="Weight")
plt.show()
sns.boxplot(data=df_events, x="Age")
plt.show()

## Apprentissage supervisé

### Thématique

Prédire avec la **taille**, le **poids** et l'**âge**, le **sport** de prédilection d'un sportif

### Préparons notre dataset

Tout d'abord, avant de commencer sur le modèle, il faut préparer les données. Il y a beaucoup de données textuelles qui représentent des classes. Comme vu dans la partie "étude des données", il faut nettoyer le dataset pour enlever toutes les abérations, valeurs manquantes et inutiles. Pour cela, nous allons filtrer les colonnes, supprimer les lignes où il manque la taille, le poids et l'âge avec `dropna` et encoder les classifications avec `LabelEncoder`.

In [ ]:
df_events = pd.read_csv("athlete_events.csv")                                   # Charge le dataset

df_events.dropna(inplace=True, subset=['Age', 'Height', 'Weight'])            # Enleve les ligne altéré
df_events = df_events[['Sex', 'Age', 'Height', 'Weight', 'Team', 'Sport']]    # Garde les cololones utiles

df_events = df_events[(df_events['Age'] < 85) & (df_events['Weight'] < 200)]  # Filtre les abération pour l'age et le poids

label_encoder_team = LabelEncoder()
df_events['Team'] = label_encoder_team.fit_transform(df_events['Team'])         # Transforme les Teams en nombres
label_encoder_sport = LabelEncoder()
df_events['Sport'] = label_encoder_sport.fit_transform(df_events['Sport'])      # Transforme les Sports en nombres
label_encoder_sex = LabelEncoder()
df_events['Sex'] = label_encoder_sex.fit_transform(df_events['Sex'])            # Transforme les Sex en nombres

df_events.head()

On peut voir ici qu'il y a une forte correlation entre le taille et le poids. De plus, on remarque aussi une petite correlation avec le sexe, la taille et le poids. On peut faire le rapprochement avec la condition physique des femmes et des hommes qui diffère.

In [ ]:
sns.heatmap(df_events.corr())
plt.show()

Ici on va préparer deux jeux de données, le premier pour entraîner le modèle, et le second pour tester si notre modèle a bien appris.

In [ ]:
Y = df_events['Sport']
X = df_events.drop('Sport', axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=True, train_size=0.8, random_state=42)

### Premier modèle: KNeighborsClassifier

Nous avons choisi comme premier modèle le KNeighborsClassifier, car son fonctionnement est assez simple. K Nearest Neighbor (KNN) est un algorithme d'apprentissage automatique très simple, facile à comprendre, polyvalent et l'un des plus utilisés pour des problèmes de classification et de régression qui sont basés sur une approche de similarité des caractéristiques.

In [ ]:
model_kn1 = KNeighborsClassifier(n_neighbors=1)
model_kn1.fit(X_train, Y_train)
print(model_kn1.score(X_train, Y_train))
accuracy = accuracy_score(Y_test,model_kn1.predict(X_test))
print('Accuracy (%): ', accuracy * 100)
print(classification_report(Y_test,model_kn1.predict(X_test)))

Dans un premier temps, nous avons décider de mettre l'hyperparamètre *n_neighbors* à 1 et nous avons trouvé un score de 94.12 ! Nous nous sommes dit que la valeur était trop haute pour un premier apprentissage. Nous avons donc choisi de prendre la racine du nombre de colonnes du dataset, soit $\sqrt{6}\approx2$. De plus, on remarque que la précision dans l'ensemble de test chute énormement.

In [ ]:
model_kn2 = KNeighborsClassifier(n_neighbors=2)
model_kn2.fit(X_train, Y_train)
model_kn2.score(X_train, Y_train)

Nous avons une précision de 44.21% et nous pouvons voir que les prédictions sont assez hasardeuses. Pour palier à cela, nous avons plusieurs options:
 - Changer de modèle
 - Classifier des données (Age, Team, Sport)
 - Normaliser les données 
 
Nous n'allons pas normaliser les données car cela n'aidera pas nos modèles.
Cependant, afin d'aider le modèle à classifier les differents types de sports, nous pouvons appliquer un encodage **one hot** pour supprimmer les relations d'ordre.

In [ ]:
accuracy = accuracy_score(Y_test,model_kn2.predict(X_test))
print('Accuracy (%): ', accuracy * 100)
print(classification_report(Y_test,model_kn2.predict(X_test)))

### Deuxième modèle AdaBoostClassifier

Nous avons pris un autre modèle de classification `AdaBoostClassifier` avec le même dataset pour comparer les performances. Les résultats sont très décevants. Le modèle n'arrive pas à prédire les différents sports comme le montre la précision des différentes classes qui est autour de 0%.

In [ ]:
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, Y_train)
clf.score(X_train,Y_train)

accuracy = accuracy_score(Y_test,clf.predict(X_test))

print('Accuracy (%): ', accuracy * 100)
print(classification_report(Y_test,clf.predict(X_test)))

### Modification du dataset pour KNeighborsClassifier

Etant donné que le premier modèle n'est pas optimal, nous avons modifié le dataset. Nous avons donc fait le filtrage habituel, mais, nous avons utilisé l'encodage **One Hot** sur toutes les catégories du dataset. Cependant, le modèle a perdu en précision, dû au nombre excessif de colones produites par le One Hot. Soit autant de colone qu'il y a de pays et de sport.

In [ ]:
df_events = pd.read_csv("athlete_events.csv")                                 # Charge le dataset

df_events.dropna(inplace=True, subset=['Age', 'Height', 'Weight'])            # Enleve les ligne useless
df_events = df_events[['Sex', 'Age', 'Height', 'Weight', 'Team', 'Sport']]    # Garde les cololones utiles

df_events = df_events[(df_events['Age'] < 85) & (df_events['Weight'] < 200)]  # Filtre les abération pour l'age et le poids

def cat_age(age):
    if 0 < age < 18: return "jeune"
    if 18 < age < 30: return "normal"
    return "vieux"

df_events['Age'] = df_events['Age'].apply(cat_age)                 # Catégorise l'age en 3 classe
df_2_X = pd.get_dummies(df_events,columns=['Sex', 'Age', 'Team'])  # Encode le Sex, Age, Team en OneHot
df_2_X = df_2_X.drop(['Sport'], axis=1)

df_2_Y = pd.get_dummies(df_events[['Sport']],columns=['Sport'])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df_2_X, df_2_Y, shuffle=True, train_size=0.8, random_state=42)

model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train, Y_train)
model.score(X_train, Y_train)

In [ ]:
accuracy = accuracy_score(Y_test,model.predict(X_test))

print('Accuracy (%): ', accuracy * 100)
print(classification_report(Y_test,model.predict(X_test)))

### Random Forest et moins de paramètres

Nous avons changé de modèle en utilisant les "Random Forest" pour minimiser la variance. Nous avons supprimer les lignes où les valeurs sur les colonnes Age, Height et Weight sont manquantes et filtrer les colonnes utiles pour éliminer les valeurs abérantes.

Nous avons choisi d'encoder la colonne Team avec des nombres pour réduire le nombre de colones obtenues après l'encodage One Hot et nous avons encodé l'âge et le sexe avec One Hot.

Ensuite, nous avons généré les trainset et le testset, puis entraîner le modèle, et nous avons réussi à passer la barre des 80% de précision ! Cependant, quand on regarde le rapport de classification, nous pouvons remarquer qu'il y a quelques incohérences sur la précision et le recall.

Pour palier à cela, nous allons classifier les différents sports pour avoir moins de classes à prédire et pour aider le modèle à se concentrer sur celles-ci.

In [ ]:
df_events = pd.read_csv("athlete_events.csv")                                 # Charge le dataset

df_events.dropna(inplace=True, subset=['Age', 'Height', 'Weight'])            # Enleve les ligne useless
df_events = df_events[['Sex', 'Age', 'Height', 'Weight', 'Team', 'Sport']]    # Garde les cololones utiles

df_events = df_events[(df_events['Age'] < 85) & (df_events['Weight'] < 200)]  # Filtre les abération pour l'age et le poids

label_encoder_team = LabelEncoder()
df_events['Team'] = label_encoder_team.fit_transform(df_events['Team'])
label_encoder_sport = LabelEncoder()
df_events['Sport'] = label_encoder_sport.fit_transform(df_events['Sport'])
label_encoder_sex = LabelEncoder()
df_events['Sex'] = label_encoder_sex.fit_transform(df_events['Sex'])

def cat_age(age):
    if 0 < age < 18: return "jeune"
    if 18 < age < 30: return "normal"
    return "vieux"

df_events['Age'] = df_events['Age'].apply(cat_age)

Y = df_events['Sport']
X = pd.get_dummies(df_events,columns=['Sex', 'Age'])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=True, train_size=0.8, random_state=42)
X.head()

In [ ]:
classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
classifier.fit(X_train, Y_train)

In [ ]:
predicted_classes = classifier.predict(X_test)
accuracy = accuracy_score(Y_test,predicted_classes)

print('Accuracy (%): ', accuracy * 100)
print(classification_report(Y_test,predicted_classes))

### Changement de thématique

Nous avons décidé de changer de thématique pour facilier de travail du modèle à classifier les différents sports tout en gardant un rapport avec la précédente problématique.

~Prédire avec la **taille**, le **poids** et l'**âge**, le **sport** de prédilection d'une personne.~

Prédire avec la **taille**, le **poids** et l'**âge**, le **type de sport (individuel ou collectif)** d'une personne.

Pour classifier les sports, nous avons récupéré tous les sports du DataFrame, enlevé les doublons et transformé cette liste en un dictionnaire associatif. De plus, on a enlevé l'encodage de l'âge car ce dernier ne faisait aucune différence. Cette solution ayant été effectuée sur les données des JO jusqu'ç 2018, elle risque de ne pas être valable pour les prochains JO, des sports pouvant être supprimer ou ajouter au fil du temps.

 - 0: sport individuel
 - 1: sport collectif

In [ ]:
df_events = pd.read_csv("athlete_events.csv")                               # Charge le dataset
sports = df_events.Sport.values.tolist()                                    # get sport value
sports = list(dict.fromkeys(sports))                                        # delete doublons 
sport_dictionary = dict.fromkeys(sports, 0)                                 # tranform to dict

def sport_to_group(sport):
    return {
     'Basketball': 1,
     'Judo': 0,
     'Football': 1,
     'Tug-Of-War': 1,
     'Speed Skating': 0,
     'Cross Country Skiing': 0,
     'Athletics': 0,
     'Ice Hockey': 1,
     'Swimming': 0,
     'Badminton': 0,
     'Sailing': 1,
     'Biathlon': 0,
     'Gymnastics': 0,
     'Art Competitions': 0,
     'Alpine Skiing': 0,
     'Handball': 1,
     'Weightlifting': 0,
     'Wrestling': 0,
     'Luge': 0,
     'Water Polo': 1,
     'Hockey': 1,
     'Rowing': 1,
     'Bobsleigh': 1,
     'Fencing': 0,
     'Equestrianism': 0,
     'Shooting': 0,
     'Boxing': 0,
     'Taekwondo': 0,
     'Cycling': 0,
     'Diving': 0,
     'Canoeing': 0,
     'Tennis': 0,
     'Modern Pentathlon': 0,
     'Figure Skating': 0,
     'Golf': 0,
     'Softball': 1,
     'Archery': 0,
     'Volleyball': 1,
     'Synchronized Swimming': 1,
     'Table Tennis': 0,
     'Nordic Combined': 0,
     'Baseball': 1,
     'Rhythmic Gymnastics': 0,
     'Freestyle Skiing': 0,
     'Rugby Sevens': 1,
     'Trampolining': 0,
     'Beach Volleyball': 1,
     'Triathlon': 0,
     'Ski Jumping': 0,
     'Curling': 1,
     'Snowboarding': 0,
     'Rugby': 1,
     'Short Track Speed Skating': 0,
     'Skeleton': 0,
     'Lacrosse': 1,
     'Polo': 1,
     'Cricket': 1,
     'Racquets': 0,
     'Motorboating': 0,
     'Military Ski Patrol': 0,
     'Croquet': 0,
     'Jeu De Paume': 0,
     'Roque': 0,
     'Alpinism': 0,
     'Basque Pelota': 1,
     'Aeronautics': 0
    }[sport]

Reprenons le code de la Random Forest avec notre nouvelle colonne *sport*. Nous avons aussi enlevé l'encodage One Hot pour l'âge et le sexe car il avait un impact négligeable sur le résultat. Dans le rapport de classification nous pouvons observer que :

#### classe 0
 - Quand on prédit une valeur, elle a 83% de chance d'être bien prédite
 - Quand on prédit une valeur, elle a 97% de chance d'être dans la bonne classe
 
#### classe 1
 - Quand on prédit une valeur, elle a 70% de chance d'être bien prédite
 - Quand on prédit une valeur, elle a 26% de chance d'être dans la bonne classe
 
Il faut noter que la classe 1 a un écart non négligeable avec le support, ses résultats ne sont donc pas représentatifs de la disparité entre les deux groupes. Pour résoudre ce déséquilibre, nous pouvons soit rajouter des éléments dans la case 1 (créer des nouveaux vecteurs) ou supprimer des vecteurs dans la classe 0. Nous choisissons de laisser la chose telle qu'elle est et d'utiliser un modèle plus performant avec xgboost pour réduire la variance. 

In [ ]:
df_events = pd.read_csv("athlete_events.csv")                               # Charge le dataset
df_events.dropna(inplace=True, subset=['Age', 'Height', 'Weight'])          # Enleve les ligne useless
df_events = df_events[['Sex', 'Age', 'Height', 'Weight', 'Team', 'Sport']]  # Garde les cololones utiles

label_encoder_team = LabelEncoder()
df_events['Team'] = label_encoder_team.fit_transform(df_events['Team'])
label_encoder_sex = LabelEncoder()
df_events['Sex'] = label_encoder_sex.fit_transform(df_events['Sex'])

df_events['Sport'] = df_events['Sport'].apply(sport_to_group)

Y = df_events['Sport']
X = df_events[["Age", "Weight", "Height", "Team", "Sex"]]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=True, train_size=0.8, random_state=42)
X_train

In [ ]:
classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
classifier.fit(X_train, Y_train)

In [ ]:
predicted_classes = classifier.predict(X_test)
accuracy = accuracy_score(Y_test,predicted_classes)

print('Accuracy (%): ', accuracy * 100)
print(classification_report(Y_test,predicted_classes))

Nous venons de tester le modèle Random Forest qui a pour but de minimiser la variance. Maintenant, nous allons essayer avec xgBoost qui va minimiser la variance avec des modèles faibles et réduire le biais.

In [ ]:
param = {
    'max_depth':4, 
    'eta':0.2, 
    'objective':'binary:logistic',
    'n_estimators': 250,
    'learning_rate': 0.05,
    'gamma': 0.1
}
booster = xgb.XGBClassifier(**param)
booster.fit(X_train, Y_train)
print(booster.score(X_train, Y_train))

In [ ]:
predicted_classes = booster.predict(X_test)
accuracy = accuracy_score(Y_test,predicted_classes)

print('Accuracy (%): ', accuracy * 100)
print(classification_report(Y_test,predicted_classes))

Maintenant que nous avons trouvé un meilleur modèle qui se focalise sur la reduction du biais, nous allons chercher les hyperparamètres optimaux de notre modèle et vérifier avec la *cross validation* si notre modèle est robuste.

Pour cela, nous utilisons `GridSearchCV` pour les hyperparamètres et `cross_val_score` pour tester la robustesse de notre modèle avec la métrique précision.


In [ ]:
param_grid = {
  'max_depth':[2, 3, 4], 
  'eta': [0.1, 0.2, 0.3], 
  'objective':['binary:logistic'],
  'n_estimators': [100, 200, 300],
  'learning_rate': [0.1, 0.05],
  'gamma': [0.1]
}

grid = GridSearchCV(xgb.XGBClassifier(), param_grid, cv=5)
grid.fit(X_train, Y_train)

In [ ]:
print(grid.best_params_)
model = grid.best_estimator_

Avec cette optimisation, nous avons gagné 1% de précision.

In [ ]:
predicted_classes = model.predict(X_test)
accuracy = accuracy_score(Y_test,predicted_classes)

print('Accuracy (%): ', accuracy * 100)
print(classification_report(Y_test,predicted_classes))

### Cross validation des données

On remarque qu'on obtient la même précision qu'avec le test set, nous pouvons donc conclure que notre modèle est donc bien robuste !

In [ ]:
scores = cross_val_score(model, X_train, Y_train, cv=5, scoring='accuracy')
scores.mean()

## Compte rendu de l'étude "Historique olympique"

### Quelle problématique globale avez-vous identifié/choisi ?

Nous avons d'abord choisi de **prédire avec la taille, le poids, l'âge et le sexe d'une personne, son sport de prédilection** puis, au vue des résultats que nous avons obtenu, nous avons changé la problématique pour **prédire si la personne est faite pour un sport individuel ou collectif**.

### Quels problèmes avez-vous rencontré avec les données ? Quelles explorations en conséquence ?

Premièrement, nous avons voulu supprimer tous les vecteurs qui avaient un *NaN* dans leurs colonnes. Cependant, nous n'avons pas enlevé les vecteurs sans médaille car nous perdisons beaucoup trop d'informations. Nous avons donc supprimé les *NaN* des vecteurs ayant pour colonnes l'âge, la taille et le poids.

Aussi, nous avons eu du mal à trouver les bons encodages pour certaines colonnes et avons finalement encoder que le sexe. Nous avons tenté d'encoder l'âge avec One Hot mais cela ne changeait pas les performances de notre modèle. De plus, nous avons tenté l'encodage One Hot sur les équipes et les sports, mais cela n'a eu que pour effet de créer un trop grand nombre colonnes et a réduit drastiquement les performances de nos modèles.

Enfin, dans nos observations, nous n'avons pas pu établir de correlation entre le type de sport et les autres attributs. En outre, nous avons remarqué que la taille et le poids étaient liés.

### Quel(s) modèle(s) avez-vous choisi ? Pourquoi ?

Nous sommes d'abord partis sur un modèle KNeighborsClassifier pour sa simplicité de compréhension. Nous avons remarqué un sur-apprentissage par une chute de performances entre la phase d'entraînement et de test. Nous avons donc changé les hyperparamètres des voisins en fonction du nombre de colonnes pour que le modèle se comporte correctement mais cela n'a pas réussi.

Pour palier à cela, nous avons choisi un autre modèle de classification AdaBoostClassifier, sans succès. Nous sommes donc revenus sur le KNeighborsClassifier avec une encodage One Hot pour minimiser les relations d'ordre. Cela a eu pour conséquences d'augmenter le temps de calcul du modèle et de réduire sa performance.

Nous avons donc décidé d'utiliser le RandomForestClassifier se basant sur les arbres pour minimiser la variance, et l'encodage numérique pour réduire le nombre de colonnes. En effet, nous avons noté une augmentation des performances par rapport aux autres modèles.  

Suite aux bonnes performances de ce genre de modèle, nous avons decidé de prendre un autre modèle d'arbre pour comparer leurs performances. Nous avons choisi xgBoost qui lui, travaille sur la reduction du biais. En utilisant le même dataset que pour les Random Forest, nous avons gagné 1% et avec les optimisations, nous avons gagné à nouveau 1%, soit un gain total de 2% entre les deux modèles.
Nous avons verfié la robustesse de notre modèle xgBoost en utilisant la crossValidation de `GridSearchCV`, avec succès. 

## Critiques

Nous pouvons bien évidemment augmenter encore ce pourcentage avec un protocole d'optimisation plus rigoureux, mais, la priorité est les données, et elles sont encore soumises à un biais énorme. 
 - Les données sont-elles correcte ?
 - Pouvons-nous réellement prédire des comportements avec les informations données ?
 - Des sports risquent d'être retirés ou ajoutés
 - Les femmes n'on pas commencé les JO en même temps que les hommes
 - Il y a deux types de Jeux Olympiques avec des sports différents
 - Il y a plus de sports individuels que collectifs

Il est donc judicieux de nuancer notre travail à travers tous ces biais et de se concentrer sur la méthodologie de notre travail.






  